In [5]:
# Answers to the questions are found below the program

import requests
from lxml import html
import re
import pandas as pd

titleList = []
pointsList = []
commentsList = []
ageList = []

def getExperiencePointsNow():
    response = requests.get('http://news.ycombinator.com/')
    page = html.fromstring(response.content)

    #Extracting title
    title = page.xpath('//a[contains(@class, "storylink")]')
    for x in title:
        titleList.append(x.text)

    #Extracting comments
    posts = page.xpath('//td[contains(@class, "subtext")]')
    for x in posts:
        #Extracting Comments
        #print x[-1].text,
        txt = re.findall('\d+', x[-1].text )
        txtNum = "".join(str(x) for x in txt)
        num = int(txtNum) if txtNum.isdigit() else 0
        commentsList.append(num)

        #Extracting Points
        #print x[0].text,
        if x[0].text:
            txt = re.findall('\d+', x[0].text )
        else:
            txt = '0'
        txtNum = " ".join(str(x) for x in txt)
        num = int(txtNum) if txtNum.isdigit() else 0
        pointsList.append(num)

        #Extracting age

        #print h[0].text
        age = x.xpath('.//span[contains(@class, "age")]/a')
        for x in age:
            #print x.text
            #Convert Hours, Days into minutes


            if 'day' in x.text or 'days' in x.text:
                txt = re.findall('\d+', x.text )
                txtNum = " ".join(str(x) for x in txt)
                num = int(txtNum)
                ageList.append(num*24*60)
            elif 'hours' in x.text or 'hours' in x.text:
                txt = re.findall('\d+', x.text )
                txtNum = " ".join(str(x) for x in txt)
                num = int(txtNum)
                ageList.append(num*60)
            else:
                txt = re.findall('\d+', x.text )
                txtNum = " ".join(str(x) for x in txt)
                num = int(txtNum)
                ageList.append(num)
    print "************Title************"
    print titleList

    print "************Comments************"
    print commentsList
    print "************Points************"
    print pointsList
    print "************Age************"
    print ageList


    #Return 2 lists: (1) top 10 articles by points, (2) top 10 articles by comments

    #Printing top 10 Comments Articles
    commentDF = pd.DataFrame(
    {'Title':titleList,
    'Comments':commentsList,
    })

    sortedcommentDF = commentDF.sort('Comments', ascending=False)
    print "Printing top 10 Articles by comments"
    print sortedcommentDF.head(10)


    #Printing top 10 Points Articles
    pointsDF = pd.DataFrame(
    {'Title':titleList,
    'Points':pointsList
    })
    sortedpointsDF= pointsDF.sort('Points', ascending=False)
    print "Printing top 10 Articles by Points"
    print sortedpointsDF.head(10)

    #Calculating average time taken for comments
    df=pd.DataFrame(
    {'Title':titleList,
    'Comments':commentsList,
    'Points':pointsList,
    'Age in Minutes':ageList})
    df['timeforcomment'] = df['Comments']/df['Age in Minutes']
    print "Average time taken for one comment in minutes is " + str(df['timeforcomment'].mean())

# Get the experience points.
getExperiencePointsNow()

'''
1. Explain how articles get ranked & pushed to frontpage of Hacker News (https://news.ycombinator.com/)
news.ycombinator.com uses a combination of number of votes and age of a post to compute it’s score. 
It is positively affected by votes and more negatively by it’s age. Following formula is used to compute the score :


Score = (V-1) / (T+2)^G

where,
V = votes of a post ( -1 to remove submitters vote)
T = age of post (in hours)
G = Gravity, default value of 1.8


This kind of a scoring considers freshness of the post more important than the number of votes hence increase 
in age of a post drastically reduces its score. Any post which attracts more votes while is still fresh gets a 
good score and ends up on frontpage. Fraudulent votes are eliminated when counting for the votes, for instance 
multiple upvotes from a single IP address etc. Any attempt to acquire votes will not make posts reach frontpage.

Return 2 lists: (1) top 10 articles by points, (2) top 10 articles by comments
Output on October 29th at 4.15 pm is the below
Printing top 10 Articles by comments
    Comments                                              Title
13       313  ESPN Loses 621,000 Subscribers; Worst Month in...
4        296           Total Nightmare: USB-C and Thunderbolt 3
16       155             How I Became HackerRank 1 in Two Hours
17       109     Why It Took So Long to Invent the Wheel (2012)
3         82  Intel driven MacBook Pros have secondary ARM p...
5         82  Thomas Piketty's Capital in the 21st Century, ...
7         79  Step-by-step tutorial to build a modern JavaSc...
10        77  A Map of the World Won Japanâs Prestigious D...
0         62                    Donât go to art school (2013)
27        56                             Academia, Love Me Back

Printing top 10 Articles by Points
    Points                                              Title
4      546           Total Nightmare: USB-C and Thunderbolt 3
13     250  ESPN Loses 621,000 Subscribers; Worst Month in...
16     196             How I Became HackerRank 1 in Two Hours
3      179  Intel driven MacBook Pros have secondary ARM p...
10     176  A Map of the World Won Japanâs Prestigious D...
5      174  Thomas Piketty's Capital in the 21st Century, ...
7      148  Step-by-step tutorial to build a modern JavaSc...
17     129     Why It Took So Long to Invent the Wheel (2012)
27     121                             Academia, Love Me Back
6      120  Photographers of 1870s London Documented Their...

3.For these 2 lists, find the Pearson correlation coefficient for these 2 lists
As discussed in the class, this question cannot be answered.

4. Ask a good question and answer it from Hacker News front page
I am analyzing the average time for comments. For this I am finding comments/age values for each article and then find
the average of the derived values
'''



************Title************
[u'Don\xe2\x80\x99t go to art school (2013)', 'Microsoft Adaptive Keyboards Concepts', u'Good News Fixing Your Car Isn\xe2\x80\x99t Illegal Anymore', 'Intel driven MacBook Pros have secondary ARM processor for Touch ID and security', 'Total Nightmare: USB-C and Thunderbolt 3', "Thomas Piketty's Capital in the 21st Century, in 20 minutes [video]", 'GiveCampus (YC S15) is hiring Ruby engineers and biz dev for educational software', 'Photographers of 1870s London Documented Their Disappearing City', 'Step-by-step tutorial to build a modern JavaScript stack from scratch', u'Frink \xe2\x80\x93 A practical calculating tool and programming language', u'A Map of the World Won Japan\xe2\x80\x99s Prestigious Design Award', 'GraphicsMagick Image Processing System', u'Dear Matt Mullenweg: An Open Letter from Wix.com\xe2\x80\x99s CEO Avishai Abrahami', u'ICANN\xe2\x80\x99s First Test of Accountability', 'Universal adversarial perturbations', 'ESPN Loses 621,000 Subscri

/Users/shraavyasirpa/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:83: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/shraavyasirpa/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:93: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


"\n1. Explain how articles get ranked & pushed to frontpage of Hacker News (https://news.ycombinator.com/)\nnews.ycombinator.com uses a combination of number of votes and age of a post to compute it\xe2\x80\x99s score. \nIt is positively affected by votes and more negatively by it\xe2\x80\x99s age. Following formula is used to compute the score :\n\n\nScore = (V-1) / (T+2)^G\n\nwhere,\nV = votes of a post ( -1 to remove submitters vote)\nT = age of post (in hours)\nG = Gravity, default value of 1.8\n\n\nThis kind of a scoring considers freshness of the post more important than the number of votes hence increase \nin age of a post drastically reduces its score. Any post which attracts more votes while is still fresh gets a \ngood score and ends up on frontpage. Fraudulent votes are eliminated when counting for the votes, for instance \nmultiple upvotes from a single IP address etc. Any attempt to acquire votes will not make posts reach frontpage.\n\nReturn 2 lists: (1) top 10 articles b